In [16]:
import pandas as pd

In [17]:
coef = pd.read_stata('../input/MIRROR_task2_output/coef.dta')
msa64_19 = pd.read_csv('../input/MIRROR_task4_output/msa64_19.csv')

pivot_coef = coef.pivot(index='year', columns='var', values='coef')

# Reset the index to make 'year' a regular column
pivot_coef.reset_index(inplace=True)


In [18]:
msa64_19.reset_index(inplace=True)

# Merge the two DataFrames based on the 'year' column
final64_19 = pd.merge(msa64_19, pivot_coef, on='year', suffixes=('', '_coef'), how='left')
final64_19 = final64_19.drop(columns = ['index', 'Unnamed: 0', '_cons'])
final64_19['hs_somecoll_collmore']  = final64_19['hs'] + final64_19['somecoll'] + final64_19['collmore']
final64_19['somecoll_collmore']  = final64_19['somecoll'] + final64_19['collmore']

In [19]:
import numpy as np
#Find residual wage
final64_19['residual'] = (final64_19['hs_somecoll_collmore'] * final64_19['highmore'] +
                            final64_19['somecoll_collmore'] * final64_19['colmore'] +
                            final64_19['female'] * final64_19['female_coef'] +
                            final64_19['nonwhite'] * final64_19['nonwhite_coef'] +
                            final64_19['age'] * final64_19['age_coef'])
final64_19['reswage'] = final64_19['logwage'] - final64_19['residual']

final64_19 = final64_19.drop(columns = ['age_coef', 'colmore', 'female_coef', 'highmore', 'nonwhite_coef', 'residual'])
final64_19['logemp'] = np.log(final64_19['emp'])
final64_19['logHP'] = np.log(final64_19['HP'])

final64_19

,msa,year,wage,meanwage,logwage,meanlogwage,emp,logemp,rent,logrent,...,female,white,nonwhite,age,total,HP,hs_somecoll_collmore,somecoll_collmore,reswage,logHP
0,40,1964,4060.505393,5213.814229,8.309063,8.550901,45153.0,10.717812,85.011783,4.442790,...,0.4991,0.9530,0.0470,28.023180,101078,1020.141393,0.567401,0.253568,7.446648,6.927697
1,120,1964,3713.002915,5213.814229,8.219596,8.550901,34300.0,10.442901,80.534773,4.388689,...,0.5082,0.6560,0.3440,25.610463,81884,966.417281,0.487245,0.204964,7.615297,6.873596
2,160,1964,5271.284413,5213.814229,8.570029,8.550901,377659.0,12.841747,97.086104,4.575598,...,0.5194,0.9674,0.0326,33.303779,737359,1165.033245,0.575410,0.236599,7.548203,7.060505
3,200,1964,4884.956515,5213.814229,8.493916,8.550901,139243.0,11.843976,99.116973,4.596301,...,0.5020,0.9659,0.0341,26.100653,315485,1189.403677,0.664669,0.325524,7.606564,7.081207
4,220,1964,3538.448647,5213.814229,8.171444,8.550901,35801.0,10.485731,72.382597,4.281966,...,0.5119,0.6946,0.3054,29.041158,111351,868.591160,0.448772,0.183419,7.476272,6.766873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,9260,2019,42715.112616,57256.461621,10.662308,10.932744,144429.0,11.880543,868.000000,6.766192,...,0.5007,0.6676,0.3324,35.200280,250649,10416.000000,0.743176,0.471287,10.049030,9.251098
822,9280,2019,44837.711576,57256.461621,10.710805,10.932744,337350.0,12.728876,972.000000,6.879356,...,0.5059,0.8408,0.1592,40.279206,447628,11664.000000,0.903240,0.511484,9.973641,9.364262
823,9320,2019,38168.179126,57256.461621,10.549757,10.932744,344472.0,12.749768,678.429736,6.519781,...,0.5101,0.8009,0.1991,42.330374,530110,8141.156831,0.903690,0.501775,9.798976,9.004688
824,9340,2019,45086.834266,57256.461621,10.716346,10.932744,65792.0,11.094254,1069.550097,6.974993,...,0.4984,0.6040,0.3960,36.745534,173839,12834.601167,0.804528,0.572163,10.024038,9.459900


In [20]:
# Calculate the employment-weighted average of log residual wage for each year
meanreswage = final64_19.groupby('year').apply(lambda x: (x['reswage'] * x['emp']).sum() / x['emp'].sum())
meanlogHP = final64_19.groupby('year').apply(lambda x: (x['logHP'] * x['emp']).sum() / x['emp'].sum())
meanHP = final64_19.groupby('year').apply(lambda x: (x['HP'] * x['emp']).sum() / x['emp'].sum())

# Reset the index to convert the result into a DataFrame
meanreswage = meanreswage.reset_index()
meanlogHP = meanlogHP.reset_index()
meanHP = meanHP.reset_index()

# Rename the columns to have meaningful names
meanreswage.columns = ['year', 'meanreswage']
meanlogHP.columns = ['year', 'meanlogHP']
meanHP.columns = ['year', 'meanHP']

# Merge the result back into the original DataFrame
final64_19 = final64_19.merge(meanreswage, on='year', how = 'inner')
final64_19 = final64_19.merge(meanlogHP, on='year', how = 'inner')
final64_19 = final64_19.merge(meanHP, on='year', how = 'inner')

print(final64_19.columns.to_list)
final64_19 = final64_19[['msa', 'year', 'wage', 'meanwage', 'logwage', 'meanlogwage', 
        'reswage', 'meanreswage', 'emp', 'logemp', 'rent', 'logrent', 'meanlogrent', 'drop', 'hs', 
        'somecoll', 'collmore', 'hs_somecoll_collmore', 'somecoll_collmore', 'male', 'female', 'white', 'nonwhite', 
        'age', 'total', 'HP', 'meanHP', 'logHP', 'meanlogHP'
]]
final64_19

final64_19.to_csv('../output/final64_19.csv')

<bound method IndexOpsMixin.tolist of Index(['msa', 'year', 'wage', 'meanwage', 'logwage', 'meanlogwage', 'emp',
       'logemp', 'rent', 'logrent', 'meanlogrent', 'drop', 'hs', 'somecoll',
       'collmore', 'male', 'female', 'white', 'nonwhite', 'age', 'total', 'HP',
       'hs_somecoll_collmore', 'somecoll_collmore', 'reswage', 'logHP',
       'meanreswage', 'meanlogHP', 'meanHP'],
      dtype='object')>


In [22]:
# Pivot
pivot64_19 = final64_19.pivot(index='msa', columns='year')
pivot64_19.columns = [f"{col[1]}{col[0]}" for col in pivot64_19.columns]
pivot64_19.reset_index(inplace=True)
pivot64_19

pivot64_19.columns = [col[4:] + col[:4] for col in pivot64_19.columns]
pivot64_19


,msa,wage1964,wage2009,wage2019,meanwage1964,meanwage2009,meanwage2019,logwage1964,logwage2009,logwage2019,...,HP2019,meanHP1964,meanHP2009,meanHP2019,logHP1964,logHP2009,logHP2019,meanlogHP1964,meanlogHP2009,meanlogHP2019
0,40,4060.505393,29304.201504,38691.262868,5213.814229,44364.517287,57256.461621,8.309063,10.285486,10.563369,...,11052.000000,1343.319902,11388.236999,14768.130498,6.927697,9.077951,9.310367,7.190779,9.315498,9.567291
1,120,3713.002915,30555.280130,39549.513354,5213.814229,44364.517287,57256.461621,8.219596,10.327293,10.585309,...,9461.985419,1343.319902,11388.236999,14768.130498,6.873596,8.987280,9.155038,7.190779,9.315498,9.567291
2,160,5271.284413,39279.277753,51364.691340,5213.814229,44364.517287,57256.461621,8.570029,10.578452,10.846706,...,12452.087612,1343.319902,11388.236999,14768.130498,7.060505,9.235511,9.429644,7.190779,9.315498,9.567291
3,200,4884.956515,36050.104377,43946.300716,5213.814229,44364.517287,57256.461621,8.493916,10.492665,10.690724,...,10901.948015,1343.319902,11388.236999,14768.130498,7.081207,9.114636,9.296697,7.190779,9.315498,9.567291
4,220,3538.448647,31241.093831,40839.971994,5213.814229,44364.517287,57256.461621,8.171444,10.349490,10.617417,...,9900.000000,1343.319902,11388.236999,14768.130498,6.766873,9.017241,9.200290,7.190779,9.315498,9.567291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,4459.251450,33534.905779,42715.112616,5213.814229,44364.517287,57256.461621,8.402736,10.420342,10.662308,...,10416.000000,1343.319902,11388.236999,14768.130498,6.930659,9.025937,9.251098,7.190779,9.315498,9.567291
272,9280,4504.762739,36538.681594,44837.711576,5213.814229,44364.517287,57256.461621,8.412891,10.506127,10.710805,...,11664.000000,1343.319902,11388.236999,14768.130498,6.972821,9.146761,9.364262,7.190779,9.315498,9.567291
273,9320,5516.502360,31400.863840,38168.179126,5213.814229,44364.517287,57256.461621,8.615499,10.354591,10.549757,...,8141.156831,1343.319902,11388.236999,14768.130498,7.082360,8.876896,9.004688,7.190779,9.315498,9.567291
274,9340,4969.140525,32662.890306,45086.834266,5213.814229,44364.517287,57256.461621,8.511002,10.393995,10.716346,...,12834.601167,1343.319902,11388.236999,14768.130498,7.124752,9.235400,9.459900,7.190779,9.315498,9.567291


In [23]:
pivot64_19.to_csv('../output/pivot64_19.csv')